In [15]:
# On importe les bibliothèque nécessaire
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [16]:
#Base de données d'entrainement
# Connexion à la base de données
conn = sqlite3.connect('train.db')
c = conn.cursor()

#Création du data frame
df_train = pd.read_sql_query("SELECT * from avis", conn)

# Validation de la transaction et fermeture de la connexion
conn.commit()
conn.close()

#Retirer des colonnes
df_train = df_train.drop('rowid', axis = 1)
df_train = df_train.drop('id', axis = 1)

#Valeurs dupliquées
df_train.duplicated().sum()
df_train.drop_duplicates(inplace=True)

#Valeurs manquantes
df_train.isnull().sum()

rating    0
text      0
dtype: int64

In [17]:
# Encodage de la colonne rating
df_train['rating'] = df_train['rating'].apply(lambda x: 0 if x < 5 else 1)

# Charger les données d'entrainement depuis la base de données
x_train = df_train['text']
y_train = df_train['rating']

# x_train est une liste de textes et y_train est une liste d'étiquettes du score
# Convertir les textes en vecteurs numériques avec la vectorisation TF-IDF
vectorizer = TfidfVectorizer()
x_train_vectorized = vectorizer.fit_transform(x_train)

# On divise les données en ensemble d'entrainement et ensemble de test
x_train_vec, x_test_vec, y_train, y_test = train_test_split(x_train_vectorized, y_train, test_size=0.2, random_state=42)

In [18]:
svm_model_best = SVC(C=1, gamma=0.1, kernel='linear')
svm_model_best.fit(x_train_vec, y_train)

y_pred_svm_b = svm_model_best.predict(x_test_vec)

# Calculer l'exactitude des prédictions
accuracy_svm_b = accuracy_score(y_test, y_pred_svm_b)
precision_svm_b = precision_score(y_test, y_pred_svm_b, average='macro')
recall_svm_b = recall_score(y_test, y_pred_svm_b, average='macro')
f1_score_svm_b = f1_score(y_test, y_pred_svm_b, average='macro')

print("Exactitude de SVM: ", accuracy_svm_b)
print("Précision de SVM: ", precision_svm_b)
print("Rappel de SVM: ", recall_svm_b)
print("F1 score de SVM: ", f1_score_svm_b)

Exactitude de SVM:  0.8859666733587633
Précision de SVM:  0.8863878031655912
Rappel de SVM:  0.8857013844652835
F1 score de SVM:  0.8858589793533308


In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.datasets import make_classification

# Création de données fictives pour l'exemple
X, y = make_classification(n_samples=1000, random_state=0)

# Création du modèle SVM
clf = SVC(kernel='linear', C=1, random_state=0)

# Validation croisée avec 5 partitions
scores = cross_val_score(clf, X, y, cv=5)

# Affichage des scores moyens et de l'écart-type
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.95 (+/- 0.02)


In [20]:
#Base de données de test
# Connexion à la base de données
conn = sqlite3.connect('test.db')
c = conn.cursor()

#Création du data frame
df_test = pd.read_sql_query("SELECT * from avis", conn)

# Validation de la transaction et fermeture de la connexion
conn.commit()
conn.close()

#Retirer des colonnes
df_test = df_test.drop('rowid', axis = 1)
df_test = df_test.drop('id', axis = 1)

#Valeurs dupliquées
df_test.duplicated().sum()
df_test.drop_duplicates(inplace=True)

#Valeurs manquantes
df_test.isnull().sum()

rating    0
text      0
dtype: int64

In [21]:
# Encodage de la colonne rating
df_test['rating'] = df_test['rating'].apply(lambda x: 0 if x < 5 else 1)

# Charger les données de test depuis la base de données
x_testb = df_test['text']
y_testb = df_test['rating']

# x_test est une liste de textes et y_test est une liste d'étiquettes du score
# Convertir les textes en vecteurs numériques avec la vectorisation TF-IDF

x_test_vectorized = vectorizer.transform(df_test['text'])

In [22]:
y_predtest_svm_b = svm_model_best.predict(x_test_vectorized)

# Calculer l'exactitude des prédictions
accuracy_svm_b = accuracy_score(y_testb, y_predtest_svm_b)
precision_svm_b = precision_score(y_testb, y_predtest_svm_b, average='macro')
recall_svm_b = recall_score(y_testb, y_predtest_svm_b, average='macro')
f1_score_svm_b = f1_score(y_testb, y_predtest_svm_b, average='macro')

print("Exactitude de SVM: ", accuracy_svm_b)
print("Précision de SVM: ", precision_svm_b)
print("Rappel de SVM: ", recall_svm_b)
print("F1 score de SVM: ", f1_score_svm_b)

Exactitude de SVM:  0.8706555922909442
Précision de SVM:  0.8707259659209727
Rappel de SVM:  0.8706784364567883
F1 score de SVM:  0.8706532296707405
